In [ ]:
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
from torch import optim
from torch.nn import functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import PIL
from torch import nn
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
df = pd.read_csv("../input/age-gender-and-ethnicity-face-data-csv/age_gender.csv")
df.head()

In [ ]:
df["pixels"] = df["pixels"].apply(lambda x : np.reshape(np.array(x.split(),dtype = "float32"),(48,48)))

In [ ]:
df.head()

In [ ]:
df.iloc[0]

In [ ]:
class dataset(Dataset):
    def __init__(self,df):
        self.df = df
        self.normalize = transforms.Normalize(mean = [0.485,0.456,0.406],std = [0.229,0.224,0.225])
    def __len__(self):
        return len(self.df)
    def __getitem__(self,ix):
        file = self.df.iloc[ix]
        age = torch.tensor(file.age / 116)
        gender = torch.tensor(file.gender).view(-1)
        eth = torch.tensor(file.ethnicity)
        im = file.pixels
        im = np.array(Image.fromarray(im).convert("RGB"))
        im = cv.resize(im,(224,224))
        im = torch.tensor(im).permute(2,0,1)
        im = self.normalize(im/255)
        
        return im.float().to(device),age.to(device),gender.float().to(device),eth.to(device)
        

In [ ]:
msk = np.random.randn(len(df)) < 0.8
train = df[msk]
test = df[~msk]

In [ ]:
train_new = dataset(train)
val_new = dataset(test)
train_loader = DataLoader(train_new,batch_size = 32,shuffle = True)
val_loader = DataLoader(val_new,batch_size = 32)

In [ ]:
im,a,b,c = next(iter(val_loader))

In [ ]:
import torchvision.models as models
model = models.vgg16(pretrained = True)

In [ ]:
for param in model.parameters():
    param.requires_grad = False

model.avgpool = nn.Sequential(
    nn.Conv2d(512,512,kernel_size = 3),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.Flatten()
)


In [ ]:
class AgeGenEthClassifier(nn.Module):
    def __init__(self):
        super(AgeGenEthClassifier,self).__init__()
        self.intermediate = nn.Sequential(
            nn.Linear(2048,512),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(512,128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128,64),
            nn.ReLU(),
        )
        self.age_classifier = nn.Sequential(
            nn.Linear(64,1),
            nn.Sigmoid()
        )
        self.gender_classifier = nn.Sequential(
            nn.Linear(64,1),
            nn.Sigmoid()
        )
        self.eth_classifier = nn.Sequential(
            nn.Linear(64,5),
            nn.Softmax()
        )
        
    def forward(self,x):
        x = self.intermediate(x)
        age = self.age_classifier(x)
        gender = self.gender_classifier(x)
        eth = self.eth_classifier(x)
        
        return age.to(device),gender.to(device),eth.to(device)
        

In [ ]:
model.classifier = AgeGenEthClassifier().to(device)

In [ ]:
model = model.to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)
age_criterion = nn.L1Loss().to(device)
gender_criterion = nn.BCELoss().to(device)
eth_criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
import time

In [ ]:
n_epochs = 5
val_gender_accuracies = []
val_age_maes = []
val_eth_accuracies = []
train_losses = []
val_losses = []
best_test_loss = 1000
start = time.time()
for epochs in range(n_epochs):
    epoch_train_loss,epoch_val_loss = 0,0
    val_age_mae,val_gender_acc,val_eth_acc,ctr = 0,0,0,0
    for ix,data in enumerate(train_loader):
        model.train()
        optimizer.zero_grad()
        im,age,gen,eth = data
        im = im.to(device)
        age = age.to(device)
        gen = gen.to(device)
        eth = eth.to(device)
        pred_age,pred_gen,pred_eth = model(im)
        age_loss = age_criterion(pred_age,age)
        gen_loss = gender_criterion(pred_gen,gen)
        eth_loss = eth_criterion(pred_eth,eth)
        total_loss = age_loss + gen_loss + eth_loss 
        total_loss.backward()
        optimizer.step()
        epoch_train_loss += total_loss.item()
    for ix,data in enumerate(val_loader):
        model.eval()
        im,age,gen,eth = data
        im = im.to(device)
        age = age.to(device)
        gen = gen.to(device)
        eth = eth.to(device)
        with torch.no_grad():
            pred_age,pred_gen,pred_eth = model(im)
        age_loss = age_criterion(pred_age,age)
        gen_loss = gender_criterion(pred_gen,gen)
        eth_loss = eth_criterion(pred_eth,eth)
        total_loss = age_loss + gen_loss + eth_loss 
        pred_gender = (pred_gen > 0.5)
        gender_acc = (pred_gender == gen).float()
        pred_eth = (pred_eth > 0.5)
        #eth_acc = (pred_eth == eth).float()
        age_mae = torch.abs(age - pred_age).float().sum()
        
        val_age_mae += age_mae
        epoch_val_loss += total_loss.item()
        #val_gender_acc += gender_acc
        #val_eth_acc += eth_acc
        ctr += len(data[0])
        
    val_age_mae /= ctr
    #val_gender_acc /= ctr
    #al_eth_acc /= ctr
    epoch_train_loss /= len(train_loader)
    epoch_val_loss /= len(val_loader)
    elapsed = time.time() - start
    best_test_loss = min(best_test_loss,epoch_val_loss)
    print('{}/{} ({:.2f}s - {:.2f}s remaining)'.format(epochs+1, n_epochs, time.time()-start, (n_epochs-epochs)*(elapsed/(epochs+1))))
    info = f'''Epoch: {epochs+1} \tTrain Loss: {epoch_train_loss:.3f} \tTest: {epoch_val_loss:.3f} \tBest Test Loss: {best_test_loss:.4f}'''
    info += f'%\t Age MAE: \ {val_age_mae:.2f}'
    print(info)
    #val_gender_accuracies.append(val_gender_acc)
    val_age_maes.append(val_age_mae)
    #val_eth_accuracies.append(val_eth_acc)

In [ ]:
!wget https://www.dropbox.com/s/6kzr8l68e9kpjkf/5_9.JPG

In [ ]:
im = cv.imread("./5_9.JPG")
im = cv.cvtColor(im, cv.COLOR_BGR2RGB)
im = cv.resize(im,(224,224)) / 255
im = torch.tensor(im).permute(2,0,1).to(device)
im = train_new.normalize(im)

In [ ]:
plt.imshow(im.permute(1,2,0).cpu().numpy())
age,gender,eth = model(im[None].float())


In [ ]:
pred_gender = gender.to('cpu').detach().numpy()
pred_age = age.to('cpu').detach().numpy()
pred_eth = eth.to('cpu').detach().numpy()

In [ ]:
pred_age * 116,pred_gender,pred_eth

In [ ]:
im,age,gen,eth = val_new[900]

In [ ]:
plt.imshow(im.cpu().permute(1,2,0).numpy())

In [ ]:
age,gen,eth

In [ ]:
age,gender,eth = model(im[None].float())

In [ ]:
age*116,gender,eth

In [ ]:
im1,age1,gen1,eth1 = val_new[990]
plt.imshow(im1.cpu().permute(1,2,0).numpy())
print(age1,gen1,eth1)

In [ ]:
age_pred,gender_pred,eth_pred= model(im1[None].float())

In [ ]:
age_pred*116,gender_pred,eth_pred

In [ ]:
!wget https://i.pinimg.com/originals/97/ed/6b/97ed6b370803649addbf66144c18c194.png

In [ ]:
im = cv.imread("./97ed6b370803649addbf66144c18c194.png")
im = cv.cvtColor(im, cv.COLOR_BGR2RGB)
im = cv.resize(im,(224,224)) / 255
im = torch.tensor(im).permute(2,0,1).to(device)
im = train_new.normalize(im)

In [ ]:
plt.imshow(im.permute(1,2,0).cpu().numpy())
age,gender,eth = model(im[None].float())

In [ ]:
age = age*116,
if gender[0] > 0.5:
    gender = "Woman"
if eth.to('cpu').detach().numpy()[0][0] > 0.5:
    eth = "White"
print(f"Age : {age[0].to('cpu').detach().numpy()[0][0]:.0f}, Gender: {gender} , Ethnicity: {eth}")